# Anomalie globale de température (NASA GISTEMP 1880-2022)

**Auteur : FERRY Frédéric (ENM/C3M) - septembre 2023**

Données : https://data.giss.nasa.gov/gistemp/

- https://data.giss.nasa.gov/gistemp/tabledata_v4/GLB.Ts+dSST.csv
- https://data.giss.nasa.gov/pub/gistemp/gistemp1200_GHCNv4_ERSSTv5.nc.gz

In [ ]:
import os

import numpy as np
import xarray as xr

import matplotlib.pyplot as plt
from matplotlib import colors

import pandas as pd
from pandas import Series
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

from cartopy import config
import cartopy.feature as cfeature
import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes
from mpl_toolkits.axes_grid1 import AxesGrid

from tqdm import tqdm

In [ ]:
iyear=pd.read_csv('./data/GLB.Ts+dSST.csv', sep=',',header=1, usecols=[0], skipfooter=1, engine='python')
tg=pd.read_csv('./data/GLB.Ts+dSST.csv', sep=',',header=1, usecols=[1,2,3,4,5,6,7,8,9,10,11,12],
               skipfooter=1, engine='python')
print(iyear)
print(tg)

months=['January','February','March','April','May','June','July',
       'August','September','October','November','December']

years=np.arange(1880,2023)
print(years)

In [ ]:
print(tg.shape)
tg2=np.array(tg).flatten()
print(tg2.shape)
dates = pd.date_range('1880-01', '2023-01', freq='M')
print(dates)
tg2 = Series(tg2, index=dates)

fig=plt.figure(figsize=(15, 10))
fig.suptitle('Global monthly temperature anomalies relative to 1951-1980', fontsize=16)
plt.title('Data : https://data.giss.nasa.gov/gistemp/tabledata_v4/GLB.Ts+dSST.csv' ,
          fontsize=12, color='grey')
plt.xlabel('Year')
plt.ylabel('Temperature anomaly (°C)')
plt.axhline(0, linewidth=1, color='k')
ax = tg2.plot(color='blue', linewidth=0.5, alpha=1)
plt.show()
figname='./figs/GISTEMP_Tanom_monthly'
fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
bounds = [-0.8,-0.6,-0.4,-0.2,-0.05,0.05,0.2,0.4,0.6,0.8,1,1.2,1.4]
color =["darkblue","blue","cyan","lightgreen","white","yellow","gold", "orange", 
         "red", "brown", "black","pink"]
cmap = colors.ListedColormap(color)
norm = colors.BoundaryNorm(bounds, cmap.N)

fig=plt.figure(figsize=(30, 4))
fig.suptitle('Global annual temperature anomalies relative to 1951-1980', fontsize=24)
ax = fig.add_subplot(111)
im=ax.imshow(tg.T, cmap=cmap, norm=norm)
cb=plt.colorbar(im, orientation="horizontal",shrink=0.75,ticks=bounds)
cb.set_label('°C')
mons=['Jan','Feb','Mar','Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
ax.set_yticks(np.arange(0,12,3))
ax.set_yticklabels(mons[::3]) #add month labels to the yaxis
ax.set_xticks(np.arange(0,len(years),10))
ax.set_xticklabels(years[::10]) #add year labels to the xaxis

xcoords = np.arange(0,len(years),10)
for xc in xcoords:
    plt.axvline(x=xc,linewidth=1, linestyle='--', color='grey')

plt.show()

figname='./figs/GISTEMP_Tanom_monthly2'
fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
print(tg2.shape)
imonth=int(input("Entrer le numéro du mois souhaité (1-12) : "))
month_data=tg2[imonth-1::12]
print(month_data.shape)

In [ ]:
fig=plt.figure(figsize=(15, 10))
fig.suptitle('Global '+months[imonth-1]+' temperature anomalies relative to 1951-1980', fontsize=16)
plt.title('Data : https://data.giss.nasa.gov/gistemp/tabledata_v4/GLB.Ts+dSST.csv' ,
          fontsize=12, color='grey')
plt.xlabel('Year')
plt.ylabel('Temperature anomaly (°C)')
plt.axhline(0, color='k')
plt.xlim(years[0],years[-1])
colormat=np.where(month_data>0, 'red','blue')
plt.bar(years,month_data, color=colormat)
xcoords = np.arange(1850,2030,10)
for xc in xcoords:
    plt.axvline(x=xc,linewidth=1, linestyle='--', color='grey')
plt.show()
figname='./figs/GISTEMP_Tanom_'+str(imonth)
fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
classement=np.array(np.argsort(month_data))[::-1]
print('Classement '+months[imonth-1]+ ' du plus chaud au plus froid : ',years[classement])

In [ ]:
tg=tg.mean(axis = 1)
print(tg)

In [ ]:
fig=plt.figure(figsize=(15, 10))
fig.suptitle('Global annual temperature anomalies relative to 1951-1980', fontsize=16)
plt.title('Data : https://data.giss.nasa.gov/gistemp/tabledata_v4/GLB.Ts+dSST.csv' ,
          fontsize=12, color='grey')
plt.xlabel('Year')
plt.ylabel('Temperature anomaly (°C)')
plt.axhline(0, color='k', linewidth=1)
plt.xlim(years[0],years[-1])
xcoords = np.arange(1880,2030,10)
for xc in xcoords:
    plt.axvline(x=xc,linewidth=1, linestyle='--', color='grey')
colormat=np.where(tg>0, 'red','blue')
plt.bar(years, tg, color=colormat)

plt.show()
figname='./figs/GISTEMP_Tglob'
fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
t    = xr.open_dataset("./data/gistemp1200_GHCNv4_ERSSTv5.nc")
print(t)
tyear=t.sel(time=slice('1880','2022')).groupby('time.year').mean('time', skipna=True)
print(tyear)

years=tyear.year.values
lat=tyear.lat.values
lon=tyear.lon.values
print(years)

data=tyear['tempanomaly']
print(data.shape)

In [ ]:
def plot_background(ax):
    ax.coastlines()
    ax.gridlines(crs=ccrs.PlateCarree(), linewidth=0.5, color='gray', alpha=0.5, linestyle='-')
    return ax

proj=ccrs.EqualEarth()

levels=np.arange(-3,3.5, 0.5)

axes_class = (GeoAxes, dict(map_projection=proj))
fig = plt.figure(figsize=(15,15))
fig.suptitle('NASA GISTEMP Temperature anomaly (°C relative to 1951-1980)', fontsize=16)

axgr = AxesGrid(fig, 111, axes_class=axes_class,
       nrows_ncols=(14, 10),
       axes_pad=0.2,
       cbar_location='bottom',
       cbar_mode='single', # None/single/each
       cbar_pad=0.2,
       cbar_size='3%',
       label_mode='')  # note the empty label_mode

start,stop = 0,len(years)-1

for i, ax in enumerate(axgr[:stop-start]):
    plot_background(ax)
    ax.set_title(str(years[i]), fontsize=10)
    p1 = ax.contourf(lon, lat, data[i,:,:], levels=levels, transform=ccrs.PlateCarree(), cmap='coolwarm', extend='both')
    #p2 = ax.contour(lon, lat, data[i,:,:], colors='black', levels=levels, linewidths=0.2, transform=ccrs.PlateCarree())
    axgr.cbar_axes[i].colorbar(p1)

plt.show()

figname='./figs/GISTEMP_Tanom'
fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
iyear=int(input("Entrer l'année souhaitée : "))
idx = list(years).index(iyear)

proj=ccrs.EqualEarth()

levels=np.arange(-3,3.5, 0.5)

fig = plt.figure(figsize=(15,10))
fig.suptitle('NASA GISTEMP Temperature anomaly (°C relative to 1951-1980)', fontsize=16)
ax = fig.add_subplot(1, 1, 1, projection=proj)
plot_background(ax)
ax.set_title(str(iyear), fontsize=14)
p1 = ax.contourf(lon, lat, data[idx,:,:], levels=levels, transform=ccrs.PlateCarree(), cmap='coolwarm', extend='both')
#p2 = ax.contour(lon, lat, data[idx,:,:], colors='black', levels=levels, linewidths=0.2, transform=ccrs.PlateCarree())
cb = fig.colorbar(p1, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05)
cb.set_label('°C', size='small')   
plt.show()

figname='./figs/GISTEMP_Tanom_'+str(iyear)
fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
for i in tqdm(range(len(years))): 
    fig = plt.figure(figsize=(15,10))
    fig.suptitle('NASA GISTEMP Temperature anomaly (°C relative to 1951-1980)', fontsize=16)
    ax = fig.add_subplot(1, 1, 1, projection=proj)
    plot_background(ax)
    ax.set_global()
    ax.set_title(str(years[i]), fontsize=14)
    p1 = ax.contourf(lon, lat, data[i,:,:], levels=levels, transform=ccrs.PlateCarree(),
                     cmap='coolwarm', extend='both')
    #p2 = ax.contour(lon, lat, data[i,:,:], colors='black', levels=levels, linewidths=0.2, transform=ccrs.PlateCarree())
    cb = fig.colorbar(p1, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05)
    cb.set_label('°C', size='small')   
    figname='./anim/GISTEMP_Tanom_'+str(years[i])
    fig.savefig(figname+'.png',bbox_inches='tight')
    plt.close()

In [ ]:
def make_animation(gif_filepath):
    from PIL import Image
    import os
    from IPython.display import Image as IPImage
    from IPython.display import display
    import time
    
    image_folder = './anim/' # répertoire contenant les fichiers PNG
    output_file = gif_filepath # nom du fichier de sortie
    animation_speed = 0.9 # vitesse de l'animation en secondes
    
    # Liste tous les fichiers PNG dans le répertoire image_folder
    files = sorted(os.listdir(image_folder))
    image_files = [f for f in files if f.endswith('.png')]
    
    # Ouvre chaque fichier PNG et ajoute l'image à une liste
    images = []
    for filename in image_files:
        img = Image.open(os.path.join(image_folder, filename))
        images.append(img)
    
    # Crée l'animation GIF
    images[0].save(output_file, save_all=True, append_images=images[1:], duration=int(animation_speed*1000), loop=0)
    # Affiche l'animation GIF dans Jupyter
    with open(output_file,'rb') as f:
        display(IPImage(data=f.read(), format='png'))
    # Efface les fichiers PNG
    for filename in image_files:
        os.remove(image_folder+filename)

In [ ]:
gif_filepath = './anim/GISTEMP_Tanom.gif'
make_animation(gif_filepath)